In [ ]:
!pip install langchain langgraph tiktoken faiss-cpu sentence-transformers langchain_community langchain_groq chromadb

In [ ]:
!pip install langchain_tavily

In [ ]:
import os
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.document_loaders import TextLoader
import os

folder_path = "/content/singapore_docs"
loaders = [TextLoader(os.path.join(folder_path, f)) for f in os.listdir(folder_path)]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(chunks, embedding)
retriever = vector_store.as_retriever()

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

llm = ChatGroq(model="gemma2-9b-it", temperature=0)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
from langchain.prompts import PromptTemplate

llm_prompt = PromptTemplate.from_template("Answer this general question: {question}")
llm_chain = llm | llm_prompt

In [ ]:
from langchain_tavily import TavilySearch
tavily = TavilySearch()
tavily.invoke("Current news about Cricket?")

{'query': 'Current news about Cricket?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://indianexpress.com/section/sports/cricket/',
   'title': 'Cricket News - The Indian Express',
   'content': 'CRICKET NEWS · Jasprit Bumrah released from India squad for fifth Test against England: BCCI · Duleep Trophy: Shardul Thakur to lead star studded West Zone team.',
   'score': 0.6337056,
   'raw_content': None},
  {'url': 'https://www.hindustantimes.com/cricket',
   'title': 'Latest Cricket News - Hindustan Times',
   'content': 'Latest Cricket News · IND vs ENG 5th Test Live Score: Akash Deep gets rid of Ben Duckett · Washington refuses to leave the crease after getting dismissed,',
   'score': 0.5925965,
   'raw_content': None},
  {'url': 'https://www.espncricinfo.com/cricket-news',
   'title': 'Cricket News | Cricket Updates | Match Report - ESPNcricinfo',
   'content': 'Stay up-to-date with the latest cricket news and updates from ESPNcricinfo. 

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_tavily import TavilySearch

tavily = TavilySearch()

rag_prompt = ChatPromptTemplate.from_template("Answer this question about Singapore tourist places based on the following context:\n{context}\nQuestion: {question}")
llm_prompt = ChatPromptTemplate.from_template("Answer this general question: {question}")
tool_prompt = ChatPromptTemplate.from_template("You asked for a tool: {question}") # This prompt is not directly used by the Tavily tool, but kept for clarity in routing logic.

# Create the individual chains
rag_chain = (
    {
        "context": RunnableLambda(itemgetter("question")) | retriever,
        "question": RunnableLambda(itemgetter("question")),
    }
    | create_stuff_documents_chain(llm, rag_prompt)
)
llm_chain = (
    RunnableLambda(itemgetter("question"))
    | llm_prompt
    | llm
)
tool_chain = (
    RunnableLambda(itemgetter("question"))
    | tavily
)

def route(input):
    if "singapore" in input["question"].lower():
        print("Routing to RAG chain")
        return "rag"
    elif "tool" in input["question"].lower() or "current" in input["question"].lower() or "weather" in input["question"].lower() or "news" in input["question"].lower():
        print("Routing to Tool chain")
        return "tool"
    else:
        print("Routing to LLM chain")
        return "llm"

# Create the router chain
router_chain = RunnableBranch(
    (lambda x: "singapore" in x["question"].lower(), rag_chain),
    (lambda x: "tool" in x["question"].lower() or "Current" in x["question"].lower() or "weather" in x["question"].lower() or "news" in x["question"].lower(), tool_chain),
    llm_chain
)

**1. Query for the RAG Chain (Singapore-specific):**

In [ ]:
response = router_chain.invoke({"question": "What are some popular tourist attractions in Singapore based on the documents?"})
print(response)

Based on the documents, here are some popular tourist attractions in Singapore:

* **National Gallery Singapore:** Southeast Asia's largest art museum housed in restored national monuments.
* **Singapore Flyer:** Asia's largest observation wheel offering panoramic views of the city.
* **Merlion Park:** Home to the iconic Merlion statue, a symbol of Singapore.
* **Marina Bay Sands:** A luxury resort and entertainment complex with a famous SkyPark observation deck and infinity pool. 


Let me know if you have any other questions about these attractions!



In [ ]:
response = router_chain.invoke({"question": "Tell me about the National Gallery Singapore based on the documents."})
print(response)

The National Gallery Singapore is Southeast Asia's largest art museum, housed in the beautifully restored former City Hall and Supreme Court buildings. 

Here are some key highlights:

* **Art Collection:** It boasts over 8,000 artworks from Singapore and the region, making it a must-visit for art lovers and history enthusiasts.
* **Global Collaborations:** The gallery collaborates with renowned museums like Tate Britain, showcasing a diverse range of artistic expressions.
* **Accessibility:** Locals enjoy free general admission, while tourists pay for special exhibitions starting at S$20.
* **Experiences:**
    * Guided tours provide insights into key artworks and techniques.
    * Interactive exhibits make it family-friendly.
    * The Singapore Night Festival is held annually at the gallery.
* **Dining:** Fine dining options like Aura are available within the gallery.
* **Location and Accessibility:** Situated at 1 St Andrew’s Road, it's easily accessible via City Hall MRT.
* **Arch

In [ ]:
response = router_chain.invoke({"question": "What are some popular tourist attractions in Singapore?"})
print(response)

Based on the text provided, here are some popular tourist attractions in Singapore:

* **National Gallery Singapore:** Southeast Asia's largest art museum housed in restored national monuments.
* **Singapore Flyer:** Asia's largest observation wheel offering panoramic views of the city.
* **Marina Bay Sands:** An iconic luxury resort and entertainment complex with a stunning SkyPark observation deck and infinity pool.
* **Merlion Park:** Home to the iconic Merlion statue, a symbol of Singapore. 


Let me know if you'd like more details about any of these attractions! 



**2. Query for the general LLM Chain:**

In [ ]:
response = router_chain.invoke({"question": "What is the highest mountain in the world?"})
print(response)

content='The highest mountain in the world is **Mount Everest**. \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 23, 'total_tokens': 38, 'completion_time': 0.027272727, 'prompt_time': 0.00132172, 'queue_time': 0.24894516600000002, 'total_time': 0.028594447}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--fe501530-92ec-46cc-ae6d-a97644418d68-0' usage_metadata={'input_tokens': 23, 'output_tokens': 15, 'total_tokens': 38}


**3. Query for the Tool Chain (Web Search):**

In [ ]:
response = router_chain.invoke({"question": "What is the current weather in London?"})
response

{'query': 'What is the current weather in London?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Weather in London',
   'url': 'https://www.weatherapi.com/',
   'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1754049586, 'localtime': '2025-08-01 12:59'}, 'current': {'last_updated_epoch': 1754048700, 'last_updated': '2025-08-01 12:45', 'temp_c': 20.0, 'temp_f': 68.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 354, 'wind_dir': 'N', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.41, 'precip_in': 0.02, 'humidity': 56, 'cloud': 50, 'feelslike_c': 20.0, 'feelslike_f': 68.0, 'windchill_c': 21.4, 'windchill_f': 70.5, 'heatindex_c': 24.3, 'heatindex_f': 75.7, 'dewpoint

In [ ]:
response = router_chain.invoke({"question": "recent news about cricket"})
response

{'query': 'recent news about cricket',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.hindustantimes.com/cricket',
   'title': 'Latest Cricket News - Hindustan Times',
   'content': 'Latest Cricket News · IND vs ENG 5th Test Live Score: Akash Deep gets rid of Ben Duckett · Washington refuses to leave the crease after getting dismissed,',
   'score': 0.6783488,
   'raw_content': None},
  {'url': 'https://indianexpress.com/section/sports/cricket/',
   'title': 'Cricket News - The Indian Express',
   'content': 'CRICKET NEWS · Jasprit Bumrah released from India squad for fifth Test against England: BCCI · Duleep Trophy: Shardul Thakur to lead star studded West Zone team.',
   'score': 0.65835214,
   'raw_content': None},
  {'url': 'https://www.espncricinfo.com/cricket-news',
   'title': 'Cricket News | Cricket Updates | Match Report - ESPNcricinfo',
   'content': 'Cricket News · Duleep Trophy: Iyer, Sarfaraz in West Zone squad; Thakur named